In [1]:
%%capture
!pip install selenium

In [2]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
SLEEP = 2
DEPTH = 200
BASE_URL = "https://iz.ru/tag/"
TOPICS = ['zdorove']

In [5]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    #subtitle: str = None
    content: str = None
    datetime: str = None

In [6]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

In [17]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            #print(driver.get(URL))
            time.sleep(SLEEP)
            scroll_pause_time = 2

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                    "document.getElementsByClassName('btn-show-more')[0].click()"
                      )
                    time.sleep(2)
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 100)"
                    )
                    time.sleep(2)
                except:

                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"itemscope": "itemscope", "itemtype": "https://schema.org/ItemList"}
            )


            items += scope.find_all("div", { 'class':'tag-materials-item__box', 'itemtype':'https://schema.org/Article'})


        except:
            pass

    return items

In [18]:
pages = get_pages()

100%|██████████| 1/1 [1:00:19<00:00, 3619.92s/it]


In [19]:
len(pages)

3216

In [12]:
pages[0]

<div class="tag-materials-item__box" data-v-ae8b9a9a="" itemprop="itemListElement" itemscope="itemscope" itemtype="https://schema.org/Article"><meta content="1" data-v-ae8b9a9a="" itemprop="position"/> <a class="tag-materials-item" data-v-ae8b9a9a="" href="https://iz.ru/1630543/2024-01-05/glavnyi-geriatr-minzdrava-nazvala-sposoby-snizheniia-riska-padenii-zimoi" itemprop="url"><meta content="/1630543/2024-01-05/glavnyi-geriatr-minzdrava-nazvala-sposoby-snizheniia-riska-padenii-zimoi" data-v-ae8b9a9a="" itemprop="mainEntityOfPage"/> <div data-v-ae8b9a9a="" itemprop="author" itemscope="itemscope" itemtype="https://schema.org/Organization"><meta content="iz.ru" data-v-ae8b9a9a="" itemprop="name"/> <meta content="https://iz.ru/" data-v-ae8b9a9a="" itemprop="url"/></div> <div data-v-ae8b9a9a="" itemprop="publisher" itemscope="itemscope" itemtype="https://schema.org/Organization"><meta content="iz.ru" data-v-ae8b9a9a="" itemprop="name"/> <div data-v-ae8b9a9a="" itemprop="logo" itemscope="item

In [20]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "tag-materials-item"})["href"]


    # article id
    s = re.findall(r"ru/\d+/", article.url)[0]
    article.id = s.rstrip('/]').lstrip('ru/')



    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article object
    soup = BeautifulSoup(html, "html.parser")

    # process article title
    title = soup.find('title').text

    #строка вида: Хуснуллин заявил... | Новости общества | Известия | 28.12.2023
    #берем текст до первого разделителя
    article.title = title[:re.search(r'\s\|', title).start()]

   # article datetime
   # берем содержимое content <meta content="2023-12-28T10:53:13+03:00" property="article:published_time"/>
    article.datetime = soup.find("meta", {'property':'article:published_time'})['content']


    # article content
    article_body = soup.find(
        'div', {'class': 'text-article__inside'})

    article.content = article_body.find('div', {'itemprop':"articleBody"}).text



    return article

In [21]:
parse_page(pages[0])

Article(id='1630543', url='https://iz.ru/1630543/2024-01-05/glavnyi-geriatr-minzdrava-nazvala-sposoby-snizheniia-riska-padenii-zimoi', title='Главный гериатр Минздрава назвала способы снижения риска падений зимой', content='\nГлавный внештатный гериатр Минздрава России Ольга Ткачева 5 января рассказала «Известиям», как снизить риск падений на улице зимой.По словам медика, существует ряд простых правил для безопасной прогулки в зимнее время. Так, перед выходом на улицу следует сделать 15–20 легких приседаний, поднимитесь на цыпочки 10–15 раз для улучшения координации.Кроме того, нужно выбирать правильную обувь с противоскользящей подошвой.«Выбирайте зимнюю обувь на резиновой подошве, чтобы поддерживать сцепление на скользких поверхностях. Прикрепите к подошве зимней обуви противоскользящие приспособления для большей устойчивости», — объяснила врач.В случае передвижения с тростью к инвентарю можно прикрепить острый наконечник или шипы для более надежной фиксации на льду. Спускаясь по лес

In [22]:

n = 0
for num_batch in range(int(len(pages)/536)):

    data = []

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("headless")
    chrome_options.add_argument("no-sandbox")
    chrome_options.add_argument("disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)

    start = n + num_batch*536
    end = start + 536
    print(start, end)
    for i in tqdm(range(start, end)):   #      [:100])):
        try:
            res = parse_page(pages[i])
            data.append(res)


        except:
            pass
    driver.close()
    name = f'/content/drive/MyDrive/IZ_parsing/IZ_{TOPICS[0]}_{start}-{end}.csv'
    pd.DataFrame(data=data).to_csv(name)



0 536


100%|██████████| 536/536 [35:46<00:00,  4.00s/it]


536 1072


100%|██████████| 536/536 [37:07<00:00,  4.15s/it]


1072 1608


...

100%|██████████| 536/536 [36:59<00:00,  4.14s/it]


2144 2680


100%|██████████| 536/536 [36:56<00:00,  4.14s/it]


2680 3216


100%|██████████| 536/536 [37:04<00:00,  4.15s/it]
